In [1]:
#Dependencies
import http.client
import pandas as pd
import json
import numpy as np
import datetime as dt
import os
import pyodbc
import openpyxls
import sqlalchemy as db

In [37]:
#Set Up Connections
########## EpiInfo ##########
conn = http.client.HTTPSConnection("epi-info-data-service.services.cdc.gov")
conn.set_debuglevel(1)
headers = {
    'authtoken': "Xuv5NBkeYElE0ICxPdTTzw==",
    'accept': "*/*"
    }
########## PDL Database ##########
#Connect to PDL database (SQL SERVER through ODBC)
pdl_engine = db.create_engine("mssql+pyodbc://@Pertussis Lab Database", use_setinputsizes=False)

# #Initiate SQLAlchemy Metadata and table objects - not sure if needed
# metadata = db.MetaData()
# coryne = db.Table(
#     'coryne', 
#     metadata,
#     autoload_with=pdl_engine
# )
# input = db.Table(
#     'Input', 
#     metadata,
#     autoload_with=pdl_engine
# )

In [38]:
#Pull in CCRF data
conn.request("POST", "/api/SurveyData/GetSurveyData?surveyid=0c8b2402-03a4-4599-824c-2bd3a0fe6ab6", headers=headers)

res = conn.getresponse()
r = res.read()
data=json.loads(r.decode("utf-8"))
ccrf_raw=pd.DataFrame(data)
pd.set_option('display.max_columns', 500)
#df[df['_Status']=='Complete']

send: b'POST /api/SurveyData/GetSurveyData?surveyid=0c8b2402-03a4-4599-824c-2bd3a0fe6ab6 HTTP/1.1\r\nHost: epi-info-data-service.services.cdc.gov\r\nAccept-Encoding: identity\r\nContent-Length: 0\r\nauthtoken: Xuv5NBkeYElE0ICxPdTTzw==\r\naccept: */*\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Wed, 13 Mar 2024 01:55:57 GMT
header: content-type: text/plain; charset=utf-8
header: server: Kestrel
header: content-length: 678349
header: set-cookie: 9765ae3492289acaddcb56a478ee7257=3e9c15f67d319b1f16d7a0d589e44bc0; path=/; HttpOnly; Secure; SameSite=None


In [4]:
#PDL Data Clean
#Select All PC IDs, add metadata
input_df = pd.read_sql_table('Input', pdl_engine)
coryne_df = pd.read_sql_table('coryne', pdl_engine)
coryne_ast_df = pd.read_sql_table('coryne_ast', pdl_engine)

In [30]:
#Clean Input
#input_df.columns
input_keep_cols=['ID','acc_num','age_lab','age_units','daterec','datecol','state_lab','spec_type','spec_type_oth','clin_summary','organization']
input_final = input_df[input_df['ID'].str.startswith('PC')][input_keep_cols].sort_values('ID',ascending=False)
#input_clean.head()

In [24]:
#Clean Coryne Results
#coryne_df.columns
coryne_keep_cols=['cdcid','toxigenic','cryne_ov_interp','coryne_pcr_interp','biotype','Coryne_biotype_oth']
coryne_clean = coryne_df[coryne_df['cdcid'].str.startswith('PC')][coryne_keep_cols].sort_values('cdcid',ascending=False)
#AST
ast_keep_cols = [col for col in coryne_ast_df if (col.endswith('_int')) or (col=='cdcid')]
#Merge Coryne with AST results
coryne_final = pd.merge(left = coryne_clean, right=coryne_ast_df[ast_keep_cols], left_on='cdcid',right_on='cdcid',how='left')
#coryne_clean.head()

In [31]:
#Merge Lab Data
pdl_df= pd.merge(left=input_final, right=coryne_final, left_on='ID',right_on='cdcid',how='left')
pdl_final= pdl_df.drop(['cdcid'],axis=1).rename({'ID':'cdcid'},axis=1)

#pdl_df.head()

 Clean CCRF - Remove dummy data (GAEXAMPLE), empty responses (state is null) and merge with PDL Data

In [95]:
ccrf_df=ccrf_raw[(ccrf_raw['RecordID'] != 'GAEXAMPLE') & (ccrf_raw['state'].notnull())]

CLean CCRF - Find duplicates with record IDs over length 2, and keep the record with complete status

In [89]:
#Grab Record ids
rids = ccrf_df['RecordID'].value_counts().to_frame().reset_index()
#Select IDs that have length over 2 and have multiple entries
dups = rids[(rids['RecordID'].str.len()>2) & (rids['count']>1)]['RecordID']
#Find Global Record ID of duplicate ID that is listed as not 'Complete'
dups_to_drop = ccrf_df[(ccrf_df['RecordID'].isin(dups)) & (ccrf_df['_Status']!='Complete')]['GlobalRecordId']
# Drop incomplete dup records   
ccrf_df_final = ccrf_df[~ccrf_df['GlobalRecordId'].isin(dups_to_drop)]

In [97]:
#Merge PDL and CCRF Data - by SPHLID



UsageError: Line magic function `%` not found.


In [ ]:
#Remove used PCIDs from list

In [26]:
ccrf_df.to_csv("ccrf.csv",index=False)